In [51]:
import requests
from bs4 import BeautifulSoup
import json
from pprint import pprint
import re

session = requests.Session()
#Ссылка на авторизацию POST запрос
link_auth = 'https://journal.ksk.expert/login/?login=yes'
#Ссылка на раздел который парсим
URL = 'https://journal.ksk.expert/articles/upravlenie-personalom/'

HEADERS = {
    'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Mobile Safari/537.36',
    'accept': '*/*'
}

data_auth = {
    'backurl': '/login/',
    'AUTH_FORM': 'Y',
    'TYPE': 'AUTH',
    'USER_LOGIN': 'maltseivan',
    'USER_PASSWORD': 'Maltseivan2021_',
    'Login': 'Войти'
}

responce = session.post(link_auth, data=data_auth, headers=HEADERS)
# Ответ сервера на POST запрос с авторизацией: print(responce)

#data = Храняться полученные статьи
data = {}



def get_html(url, params=None):
    r = session.get(url, headers=HEADERS, params=params)
    return r
def get_content(html):
    #Парсер ссылок раздела
    soup = BeautifulSoup(html, 'html.parser')
    link = soup.find_all('a', class_='link-article')
    cat = soup.find('h1', class_='ch-title').text
    #Раздел в котором парсим pprint(cat)
    
    for elem in link:
        link_href_attr = elem.get("href")
        urlink = 'https://journal.ksk.expert' + link_href_attr
        
        #Парсер статьи по ссылкам
        def get_html_article(urlink, params=None):
            r_text = session.get(urlink, headers=HEADERS, params=params)
            return r_text

        def get_content_article(html):
            soup = BeautifulSoup(html, 'html.parser')
            chapter_article = soup.find('h1', class_='ch-title').text
            content_article = soup.find_all('div', class_='contents mb-60')
            #Название статей в разделе pprint(chapter_article)

            for elem in content_article:
                index_text = elem.text
                #Контентая часть статьи pprint(index_text[0])
                data[chapter_article] = [index_text]
                
                with open(cat + '.json', 'w', encoding='utf-8') as file:
                    json.dump(data, file, indent=3, ensure_ascii=False)

        def parse_article():
            result = get_html_article(urlink)
            if result.status_code == 200:
                get_content_article(result.text)
            else:
                print('Не могу получить страницу!')
         
        parse_article();
        

def parse():
    result = get_html(URL)
    if result.status_code == 200:
        get_content(result.text)
    else:
        print('Не могу получить страницу!')


parse();
